In [ ]:
import os
import pandas as pd
import numpy as np

folder_path = "/mnt/B-SSD/maltamed/SAM-zero-shot/code/all_dice"

for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        # Read the CSV file
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)

        # Add a new blank row at the end
        df = pd.concat([df, pd.DataFrame([np.nan], columns=df.columns)], ignore_index=True)

        # Add a row with the mean of each column
        mean_values = df.mean()
        df = pd.concat([df, mean_values.to_frame().T], ignore_index=True)

        # Compute the mean of the highest values in each group and add it to the bottom row
        group_names = ["dice_rc", "dice_rs3", "dice_rs5", "dice_cp", "dice_bb", "dice_bbs_05", "dice_bbs_1", "dice_bbs_2"]
        highest_means = []

        for group_name in group_names:
            columns = [f"{group_name}_{i}" for i in range(3)]
            max_values = df[columns].max(axis=1)
            highest_mean = max_values.mean()
            highest_means.append(highest_mean)

        highest_means_series = pd.Series(highest_means, index=[f"{group_name}_mean" for group_name in group_names])
        df = pd.concat([df, highest_means_series.to_frame().T], ignore_index=True)

        # Save the modified DataFrame to a new CSV file with prefix "modified_"
        modified_filename = f"modified_{filename}"
        modified_file_path = os.path.join(folder_path, modified_filename)
        df.to_csv(modified_file_path, index=False)

In [2]:
import numpy as np
import os
import pandas as pd

folder_path = "/mnt/B-SSD/maltamed/SAM-zero-shot/code/all_dice"

for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        #check if there is already a json file
        if os.path.isfile(f'{filename[:-4]}.json'):
            continue
        # Read the CSV file
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        
        #remove the image column
        df = df.drop(columns=['image'])
        #remove the first column
        df = df.drop(df.columns[0], axis=1)
        #remove brackets from the whole df
        df = df.replace('\[','',regex=True)
        df = df.replace('\]','',regex=True)
        
        #convert all values to float
        df = df.astype(float)
        
        #for each column, calculate the mean

        # Compute the mean of the highest values in each group and add it to the bottom row
        group_names = ["dice_rc", "dice_rs3", "dice_rs5", "dice_cp", "dice_bb", "dice_bbs_05", "dice_bbs_1", "dice_bbs_2"]
        highest_means = []
        

        for group_name in group_names:
            columns = [f"{group_name}_{i}" for i in range(3)]
            
            df[f'{group_name}_max'] = df.apply(lambda x: x[columns].max(), axis=1)
            
        # get the mean of all columns in a json
        means = df.mean().to_json(f'{filename[:-4]}.json')
         
        # Save the modified DataFrame to a new CSV file with prefix "modified_"
        # modified_filename = f"modified_{filename}"
        # modified_file_path = os.path.join(folder_path, modified_filename)
        # df.to_csv(modified_file_path, index=False)